In [1]:
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
import requests
import folium

In [2]:
url='https://raw.githubusercontent.com/Fidaeic/IBM-capstone/master/Centroides_Barrios_Malaga.json'
archivo = requests.get(url).json()

In [3]:
malaga_datos=archivo['features']

In [4]:
barrios=pd.DataFrame()
for data in malaga_datos:
    id_barrio  = data['properties']['NUMBARRIO'] 
    nombre_barrio = data['properties']['NOMBARRIO']
        
    latlon = data['geometry']['coordinates']
    lat = latlon[1]
    lon = latlon[0]
    
    barrios = barrios.append({'Num_barrio': id_barrio,
                                          'Nombre': nombre_barrio.lower().capitalize(),
                                          'Latitud': lat,
                                          'Longitud': lon}, ignore_index=True)
barrios["Num_barrio"]=barrios["Num_barrio"].map(lambda x: int(x))
barrios.head()

,Latitud,Longitud,Nombre,Num_barrio
0,36.651810,-4.514196,Cañada de ceuta,307
1,36.662529,-4.513345,San juan-el albaricocal,321
2,36.735346,-4.427377,La rosaleda,109
3,36.739066,-4.428161,503 viviendas,378
4,36.740912,-4.429205,La palmilla,107


In [5]:
CLIENT_ID = 'MQ2VBTWEJB22CCJS4SLRXEOZPXQ5GOUMQXOCSNW5Z02V1RV4' # your Foursquare ID
CLIENT_SECRET = 'MJ43MP3Y1J5ZQLSNQP2UVWWMF3N05YIYUR1ORYBOJZTKUJ2J' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

def getNearbyFoodPlaces(names, latitudes, longitudes, radius=500, LIMIT=100):
    categoria='4d4b7105d754a06374d81259'
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)

         # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
                CLIENT_ID, 
                CLIENT_SECRET, 
                VERSION, 
                lat, 
                lng, 
                radius, 
                LIMIT,
                categoria)
        try:
                
            results = requests.get(url).json()["response"]['groups'][0]['items']

            venues_list.append([(
                name, 
                lat, 
                lng,
                v['venue']['id'],
                v['venue']['name'], 
                v['venue']['location']['lat'], 
                v['venue']['location']['lng'],  
                v['venue']['categories'][0]['name']) for v in results])
        except:
            pass

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Barrio', 
                      'Latitud_barrio', 
                      'Longitud_barrio',
                      'IdRest',
                      'Lugar', 
                      'Latitud_lugar', 
                      'Longitud_lugar', 
                      'Categoría']
        
    
    return(nearby_venues)

In [6]:
malaga_jama = getNearbyFoodPlaces(names=barrios['Nombre'],
                                   latitudes=barrios['Latitud'],
                                   longitudes=barrios['Longitud']
                                  )

Cañada de ceuta
San juan-el albaricocal
La rosaleda
503 viviendas
La palmilla
Virreina
Herrera oria
Virreina alta
26 de febrero
Olletas
Jardin de la abadia
Haza del campillo
La consula
Seminario
Ensanche centro
Carranque
Cruz del humilladero
Parque las virreinas
El coronel
Campo de golf
Pizarrillo
Sierra blanquilla
Jardin de malaga
El palo
La cizaña
Parq emp perez texeira
Recinto ferial cortijo torres
Parq emp la estrella
Sagrada familia
Las erizas
Las cuevas
Miramar del palo
La pelusa
Centro historico
La merced
San felipe neri
Capuchinos
Camino de olias
El ejido
Lagunillas
Vega de oro
Campamento benitez
Miraflores
Playa virginia
Pol alameda
Cortijo de maza
Los chopos
Las niñas
Carambuco
Hacienda sanchez blanca
La estacion
La malagueta
Perchel sur
La aurora
Los tilos
Nucleo gral franco
4 de diciembre
Ntra sra del carmen
Los prados
La victoria
Nuevo san andres 1
Campos eliseos
Monte de gibralfaro
Pol ctra de cartama
La barriguilla
La pelusilla
Explanada de la estación
Finca el candado
L

In [7]:
malaga_jama.head()

,Barrio,Latitud_barrio,Longitud_barrio,IdRest,Lugar,Latitud_lugar,Longitud_lugar,Categoría
0,Cañada de ceuta,36.651810,-4.514196,4b5b115af964a520e9e128e3,Pizzorante Bellavista,36.652013,-4.512791,Pizza Place
1,Cañada de ceuta,36.651810,-4.514196,4be6ac16bcef2d7f365805e5,Los Pinos del Coto,36.650421,-4.517435,Restaurant
2,San juan-el albaricocal,36.662529,-4.513345,4cc2d74001fb236a3d6ba1ba,La Cónsula,36.663345,-4.508297,Mediterranean Restaurant
3,San juan-el albaricocal,36.662529,-4.513345,4d73a20e946b6dcb77d44e7a,La Taberna del Panadero,36.661197,-4.514749,BBQ Joint
4,San juan-el albaricocal,36.662529,-4.513345,5a771adc9ba3e5792abdba55,Mesón Asador Viñolo,36.661131,-4.514523,BBQ Joint


In [8]:
malaga_jama.shape

(3639, 8)

In [9]:
print('There are {} uniques categories.'.format(len(malaga_jama['Categoría'].unique())))

There are 56 uniques categories.


In [10]:
malaga_onehot = pd.get_dummies(malaga_jama[['Categoría']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
malaga_onehot['Barrio'] = malaga_jama['Barrio'] 

# move neighborhood column to the first column
fixed_columns = [malaga_onehot.columns[-1]] + list(malaga_onehot.columns[:-1])
malaga_onehot = malaga_onehot[fixed_columns]

malaga_onehot.head()

,Barrio,American Restaurant,Argentinian Restaurant,Asian Restaurant,BBQ Joint,Bakery,Bistro,Brazilian Restaurant,Breakfast Spot,Buffet,...,Snack Place,South American Restaurant,Spanish Restaurant,Steakhouse,Sushi Restaurant,Taco Place,Tapas Restaurant,Thai Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant
0,Cañada de ceuta,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Cañada de ceuta,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,San juan-el albaricocal,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,San juan-el albaricocal,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,San juan-el albaricocal,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
malaga_grupo = malaga_onehot.groupby('Barrio').mean().reset_index()
malaga_grupo.head()

,Barrio,American Restaurant,Argentinian Restaurant,Asian Restaurant,BBQ Joint,Bakery,Bistro,Brazilian Restaurant,Breakfast Spot,Buffet,...,Snack Place,South American Restaurant,Spanish Restaurant,Steakhouse,Sushi Restaurant,Taco Place,Tapas Restaurant,Thai Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant
0,25 años de paz,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.500000,0.0,0.0,0.0,0.0,0.0,0.166667,0.0,0.0,0.0
1,26 de febrero,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.285714,0.0,...,0.142857,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
2,4 de diciembre,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.2,0.0,0.0,0.0,0.400000,0.0,0.0,0.0
3,503 viviendas,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
4,720 viviendas,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.100000,0.0,0.0,0.0


In [12]:
from sklearn.cluster import KMeans
import folium

In [13]:
# set number of clusters
kclusters = 10

malaga_grupo_clustering = malaga_grupo.drop('Barrio', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(malaga_grupo_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 2, 3, 0, 3, 8, 3, 0, 8, 4], dtype=int32)

In [14]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [15]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Barrio']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Barrio'] = malaga_grupo['Barrio']

for ind in np.arange(malaga_grupo.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(malaga_grupo.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Barrio,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,25 años de paz,Snack Place,Café,Tapas Restaurant,Restaurant,Vegetarian / Vegan Restaurant
1,26 de febrero,Breakfast Spot,Italian Restaurant,Snack Place,Mediterranean Restaurant,Diner
2,4 de diciembre,Tapas Restaurant,Spanish Restaurant,Burger Joint,Café,Vegetarian / Vegan Restaurant
3,503 viviendas,Mediterranean Restaurant,Fast Food Restaurant,Seafood Restaurant,Burger Joint,Vegetarian / Vegan Restaurant
4,720 viviendas,Café,Fast Food Restaurant,Pizza Place,Tapas Restaurant,BBQ Joint


In [16]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [17]:
malaga_merged=barrios.set_index("Nombre").join(neighborhoods_venues_sorted.set_index("Barrio")).reset_index().rename({"index":"Barrio"},axis=1).dropna()
malaga_merged["Cluster Labels"]=malaga_merged["Cluster Labels"].map(lambda x: int(x))
malaga_merged.head()

,Barrio,Latitud,Longitud,Num_barrio,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,25 años de paz,36.701313,-4.440729,162,3,Snack Place,Café,Tapas Restaurant,Restaurant,Vegetarian / Vegan Restaurant
1,26 de febrero,36.741873,-4.428834,106,2,Breakfast Spot,Italian Restaurant,Snack Place,Mediterranean Restaurant,Diner
2,4 de diciembre,36.713667,-4.444871,136,3,Tapas Restaurant,Spanish Restaurant,Burger Joint,Café,Vegetarian / Vegan Restaurant
3,503 viviendas,36.739066,-4.428161,378,0,Mediterranean Restaurant,Fast Food Restaurant,Seafood Restaurant,Burger Joint,Vegetarian / Vegan Restaurant
4,720 viviendas,36.736574,-4.429669,379,3,Café,Fast Food Restaurant,Pizza Place,Tapas Restaurant,BBQ Joint


In [18]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map

latitude, longitude = (36.717448, -4.431831)

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(malaga_merged['Latitud'], malaga_merged['Longitud'], malaga_merged["Barrio"], malaga_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [51]:
cluster_frec = malaga_merged.set_index(["Barrio","Cluster Labels"]).count(level="Cluster Labels")[["Latitud"]]
cluster_frec.columns=["Frequency"]

In [53]:
cluster_frec.sort_values(by="Frequency", ascending=False)

,Frequency
Cluster Labels,
8,111
3,83
6,37
4,32
0,30
5,26
2,24
9,24
1,13


In [54]:
malaga_merged[malaga_merged["Cluster Labels"]==8].head()

,Barrio,Latitud,Longitud,Num_barrio,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
5,Aeropuerto base aerea,36.678051,-4.495256,331,8,Fast Food Restaurant,Café,Spanish Restaurant,Tapas Restaurant,Cafeteria
8,Almudena,36.686885,-4.450298,198,8,Café,Tapas Restaurant,Spanish Restaurant,Restaurant,Burger Joint
11,Ardira,36.696803,-4.450937,178,8,Spanish Restaurant,Café,Pizza Place,Restaurant,Seafood Restaurant
13,Arroyo de los angeles,36.730668,-4.432975,112,8,Spanish Restaurant,Café,Restaurant,Breakfast Spot,Fast Food Restaurant
15,Arroyo españa,36.752082,-4.480916,265,8,Mediterranean Restaurant,Vegetarian / Vegan Restaurant,German Restaurant,Fried Chicken Joint,French Restaurant


In [55]:
malaga_merged[malaga_merged["Cluster Labels"]==3].head()

,Barrio,Latitud,Longitud,Num_barrio,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,25 años de paz,36.701313,-4.440729,162,3,Snack Place,Café,Tapas Restaurant,Restaurant,Vegetarian / Vegan Restaurant
2,4 de diciembre,36.713667,-4.444871,136,3,Tapas Restaurant,Spanish Restaurant,Burger Joint,Café,Vegetarian / Vegan Restaurant
4,720 viviendas,36.736574,-4.429669,379,3,Café,Fast Food Restaurant,Pizza Place,Tapas Restaurant,BBQ Joint
6,Alaska,36.702562,-4.441371,161,3,Snack Place,Tapas Restaurant,Burger Joint,Café,Vegetarian / Vegan Restaurant
14,Arroyo del cuarto,36.720133,-4.441215,113,3,Burger Joint,Café,Seafood Restaurant,Spanish Restaurant,Breakfast Spot


## Drop less than 50 venues

In [7]:
condicion=malaga_jama.groupby("Barrio", as_index=False).count().sort_values("Categoría", ascending=False)
condicion=condicion[condicion["Categoría"]>30]

In [8]:
mlg_merge=pd.merge(malaga_jama, condicion, how='inner', on='Barrio')
mlg_merge.drop(["Latitud_barrio_y","Longitud_barrio_y","IdRest_y","Lugar_y","Latitud_lugar_y","Longitud_lugar_y","Categoría_y"] ,axis=1, inplace=True)
mlg_merge.head()

,Barrio,Latitud_barrio_x,Longitud_barrio_x,IdRest_x,Lugar_x,Latitud_lugar_x,Longitud_lugar_x,Categoría_x
0,Ensanche centro,36.717007,-4.42059,5690d277498e979c7ef1b953,Santa Canela,36.716856,-4.423319,Breakfast Spot
1,Ensanche centro,36.717007,-4.42059,58e40f20326c5a26e154a24c,Kirei,36.717125,-4.420753,Sushi Restaurant
2,Ensanche centro,36.717007,-4.42059,557ec82a498e1a9642eb7421,Nacalú,36.719370,-4.420385,Tapas Restaurant
3,Ensanche centro,36.717007,-4.42059,4c25e727136d20a13754e461,Marisquería Los Mellizos,36.718753,-4.420868,Seafood Restaurant
4,Ensanche centro,36.717007,-4.42059,4c68563531ba2d7f8f85f772,La Reserva 12,36.719314,-4.420329,Spanish Restaurant


In [9]:
nombres= malaga_jama.columns

In [10]:
mlg_merge.columns=nombres

## Dividimos la muestra en 3 partes

In [11]:
mlg_merge1=mlg_merge[:499]
mlg_merge2=mlg_merge[499:999]
mlg_merge3=mlg_merge[999:]

In [12]:
def reviews(names, latitudes, longitudes, idRest, client_id, client_secret):
    version='20180605'
    
    venues_list=[]
    for name, lat, lng, venue_id in zip(names, latitudes, longitudes, idRest):
        print([name, venue_id])
        
        url = 'https://api.foursquare.com/v2/venues/{}?&v={}&client_id={}&client_secret={}'.format(
                    venue_id,
                    version,
                    client_id,
                    client_secret)

        try:

            results = requests.get(url).json()["response"]['venue']

            venues_list.append([(
                        name, 
                        lat, 
                        lng,
                        results['id'],
                        results['name'], 
                        results['location']['lat'], 
                        results['location']['lng'],
                        results['rating'],
                        results['likes']['count'],
                        results['categories'][0]['name'])])

        except:
            pass

        rated_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        rated_venues.columns = ['Barrio', 
                              'Latitud_barrio', 
                              'Longitud_barrio',
                              'IdRest',
                              'Lugar', 
                              'Latitud_lugar', 
                              'Longitud_lugar',
                              'Nota',
                              'Likes',
                              'Categoría']


    return(rated_venues)

In [20]:
CLIENT_ID1 = 'MQ2VBTWEJB22CCJS4SLRXEOZPXQ5GOUMQXOCSNW5Z02V1RV4'
CLIENT_SECRET1 = 'MJ43MP3Y1J5ZQLSNQP2UVWWMF3N05YIYUR1ORYBOJZTKUJ2J'

CLIENT_ID2 = 'TCKBXIKDGZOZWGI1QZOHLFKSI03U2NQ5ORN53Q1FQVFLWF3X' 
CLIENT_SECRET2 = 'U1VXG3TTTMZSRNZDOQ4INQCYE512MXJAOOAM1LCSF3JQ1DZI' 

CLIENT_ID3='YCPTXQARFBU5Z0KLXIFBXFUOGPLFESUKVNLLX5BAGPPIN35H'
CLIENT_SECRET3='XQAUEBF2DGILYJMDSPS24JC3NEFONEHMWKSIPYORK42CBAS2'

CLIENT_ID4 ='XY1HGAGLWX0NMBNB3WTTTDX4LFQ0SULQYDPXRWMZIG0I4Q20'
CLIENT_SECRET4 ='OUJB1T33IZAK2Q3LS1QUD2ZRGAGXVSJLIZNM5P0GAGC4FSCB'

In [16]:
malaga_notas1 = reviews(mlg_merge1['Barrio'],
                       mlg_merge1['Latitud_barrio'],
                       mlg_merge1['Longitud_barrio'],
                       mlg_merge1["IdRest"],
                       CLIENT_ID1,
                       CLIENT_SECRET1)

['Ensanche centro', '5690d277498e979c7ef1b953']
['Ensanche centro', '58e40f20326c5a26e154a24c']
['Ensanche centro', '557ec82a498e1a9642eb7421']
['Ensanche centro', '4c25e727136d20a13754e461']
['Ensanche centro', '4c68563531ba2d7f8f85f772']
['Ensanche centro', '4b9cbd43f964a5201b7936e3']
['Ensanche centro', '596cf0771fa763356791342a']
['Ensanche centro', '4b767cf3f964a520994e2ee3']
['Ensanche centro', '51407a76e4b00ccf35393df7']
['Ensanche centro', '4e26d382d16486291ba3a75d']
['Ensanche centro', '54143d6d498e60ee78c786e9']
['Ensanche centro', '542010f2498ec006c4128241']
['Ensanche centro', '563688d6498efb2f4dc49a58']
['Ensanche centro', '4b7f0c23f964a520a71230e3']
['Ensanche centro', '571b6c38498eba1f33b72f74']
['Ensanche centro', '5674aaa8498eae8c7d6df549']
['Ensanche centro', '51c9f978498e83130db4b720']
['Ensanche centro', '50b3bf6ce412f944ee908ff0']
['Ensanche centro', '4c837ad5e602b1f7bf15a67a']
['Ensanche centro', '4b55c31ff964a520daee27e3']
['Ensanche centro', '4c40d74a520fa5932cb

['La merced', '4c8513ffee6fef3bde9a395c']
['La merced', '5a43f52b7918710289236f3b']
['La merced', '4d5fd5625c39b1f79a75fc49']
['La merced', '537370da498ebf270bb1b4f5']
['La merced', '520f915211d2d7de4236819a']
['La merced', '57a49562498effbd324dfb0e']
['La merced', '4c2ba8ced1a10f475a4bf864']
['La merced', '4beb03f262c0c9285896e1d4']
['La merced', '4e469687887781a619ac0fe4']
['La merced', '57e43fddcd102cce8bfaa4cf']
['La merced', '563d07a6cd106e704956eb26']
['La merced', '58a0d92c72714f7d353e1407']
['La merced', '4b55c31ff964a520daee27e3']
['La merced', '4cfc16b3c6cca35de8d68f32']
['La merced', '4bb661a7f562ef3b65113097']
['La merced', '4dd6ba2852b1a5c6443b0b97']
['La merced', '57af0d0f498e444c99dcdb87']
['La merced', '4c363ce993db0f47b1ba1d92']
['La merced', '5131fe32e4b0ba071cc53e0c']
['La merced', '577183b9498ef7463bdc6707']
['La merced', '4b6f2dbef964a52058e22ce3']
['La merced', '554d1d6d498eafc260542ab4']
['La merced', '4b8bcc8cf964a520f2ab32e3']
['La merced', '50cb372de4b05a8a0ee

['Lagunillas', '4d023653582454816cc449ac']
['Lagunillas', '547e0b2a498e29ea03ddd89a']
['Lagunillas', '4fb6535ee4b03d283440fb40']
['Lagunillas', '59e5231f3c858d1155427741']
['Lagunillas', '4bbb4bddb35776b0e7c4cb01']
['Lagunillas', '4c12b91aa9c220a17a86549d']
['Lagunillas', '4ccf1c77aa25a35d7170240f']
['Lagunillas', '5827873627e8a32a798c4f57']
['Lagunillas', '4c3771dc2c8020a1ce088900']
['Lagunillas', '57da8a23cd10b94f031301dc']
['Lagunillas', '5769bbf5498e473722f2f12b']
['Lagunillas', '56ee8e4d498e9e1101acb56b']
['Lagunillas', '4d0a0fb8c370a35df9850e22']
['Lagunillas', '4da43d8d7aee54811230f9fe']
['Lagunillas', '5960e6c059c4234a4c049cc5']
['La malagueta', '4f4cf2ece4b0577d80283d45']
['La malagueta', '554a3a86498e315a27eba3f6']
['La malagueta', '51af9a7a498e107fcea3d982']
['La malagueta', '5777cfb9498ec5cc170608b9']
['La malagueta', '5800e82b38fad39b2b1b7990']
['La malagueta', '4c052b7a187ec928fa01b87b']
['La malagueta', '5206500c2fc6f10bed61b77d']
['La malagueta', '4bce2c58ef1095215da683

In [21]:
malaga_notas2 = reviews(mlg_merge2['Barrio'],
                       mlg_merge2['Latitud_barrio'],
                       mlg_merge2['Longitud_barrio'],
                       mlg_merge2["IdRest"],
                       CLIENT_ID4,
                       CLIENT_SECRET4)

['Los tilos', '4dfe5d18aeb7ea26f9abb036']
['Los tilos', '4c4ae3735609c9b637df5490']
['Los tilos', '524ad373498ec93429dc3ced']
['Los tilos', '4c7129af344437042512265f']
['Los tilos', '4bb88c881261d13a1db0e898']
['Los tilos', '4bb503a20fb7b713f7d2bf8a']
['Los tilos', '4e876da85c5cfd1dcc657cea']
['Los tilos', '52944e31498e7e1cc6e20543']
['Los tilos', '4cb027f6cbab236a6e229e73']
['Los tilos', '4f6630cee4b0fb96f7635ee1']
['Los tilos', '4ef337e949010b41f66825bf']
['Los tilos', '4c7822ea794e224b1e266128']
['Los tilos', '4bdecaf0ffdec9285a9ae9a1']
['Los tilos', '4c0a81f9009a0f479b7be9bf']
['Los tilos', '4c59662404f9be9a2b20ef60']
['Los tilos', '53c7b138498edeb14aeb833f']
['Los tilos', '56812806498e639d229e20e4']
['Los tilos', '4bffe07bf61ea59367e3ea13']
['Los tilos', '57a5fb23498ec1f9eaff9370']
['Los tilos', '509eb5c6e4b076197c36fbb2']
['Los tilos', '50410dd6e4b0805b9ecf3101']
['Los tilos', '4cea7f080f196dcb142052ae']
['Los tilos', '4d8c8c4dc1b1721ee0e20246']
['Los tilos', '4e496d828130d6443ed

['Las acacias', '4e048a29b61c7592ae6faaa4']
['Las acacias', '4c002eeb92a6c928eecb43e2']
['Las acacias', '4ca790a7d971b1f752a50be1']
['Las acacias', '4ca78d1db0b8236a9da9aae6']
['Las acacias', '50183911e4b049d39482316a']
['Las acacias', '4c14d61ca5eb76b0b249c2b7']
['Las acacias', '4d974524af3d236a1fbf1ec7']
['Las acacias', '4cbd9e41d78f46886b84ce73']
['Las acacias', '4bdc17b763c5c9b6a3602a68']
['Las acacias', '4c77b36f923ba143ddc267e6']
['Las acacias', '4c095a5a7e3fc92886d4f182']
['Las acacias', '4d7a4e1eecf7a35d7a7d86f4']
['Las acacias', '51d169b0498ec2e47390978a']
['Las acacias', '4caf9a9bef1b370450cd3300']
['Las acacias', '4c082ff7bbc676b0f22846d5']
['Las acacias', '52c4215b11d2ac4fe3f760b8']
['Las acacias', '4c8789f5d92ea09300b97672']
['Las acacias', '4e10cdd9483bee47ff2cae0c']
['Las acacias', '55fa7928498e426703540c01']
['Las acacias', '5211345e11d25c859818756b']
['Las acacias', '539d9b78498e08a9ddc61aaf']
['Las acacias', '4b9be026f964a5202f3036e3']
['Las acacias', '56e46fa6498eebc

['Santa paula', '4cfa35bd2d80a143141843d8']
['Santa paula', '4c683db95fae20a1caff48a6']
['Santa paula', '50e0d476e4b01580a929c486']
['Santa paula', '558b1298498e5c83b1ce1ece']
['Santa paula', '51f6cea4498ed573672b0af8']
['Santa paula', '51d6c864498ed970997130e6']
['Santa paula', '4dbbd591cda109aa6c8b5dbb']
['Santa paula', '4d8f35f51716a143c5c244f7']
['Santa paula', '4bf7cee28d30d13a2e88ff17']
['Santa paula', '4d91ba10270e6ea8d0e858ff']
['Santa paula', '4e3d39de1838cb1b82ecf28c']
['Santa paula', '4e45aaf77d8b91a06584bd40']
['Santa paula', '4c42ecbcff711b8d15341405']
['Santa paula', '51201367e4b0b8daea4aab4c']
['Santa paula', '4d67b4f12433a14344364fe0']
['Santa paula', '4d749bfc0a22236ab0cc3702']
['Santa paula', '4c65a7aaaebea59328ff70d0']
['Santa paula', '5309f250498e04ed32f8368b']
['Santa paula', '4bb4524249bdc9b60cb70c10']
['Plaza de toros vieja', '4d97935ae07ea35d207bdd02']
['Plaza de toros vieja', '4d2bedb8ebacb1f777571050']
['Plaza de toros vieja', '50b61c9ce4b0be8d47ed47ca']
['Pla

In [19]:
malaga_notas3 = reviews(mlg_merge3['Barrio'],
                       mlg_merge3['Latitud_barrio'],
                       mlg_merge3['Longitud_barrio'],
                       mlg_merge3["IdRest"],
                       CLIENT_ID3,
                       CLIENT_SECRET3)

['Los guindos', '4e3d39de1838cb1b82ecf28c']
['Los guindos', '4d91ba10270e6ea8d0e858ff']
['Los guindos', '4c94b48703413704dc3e74ef']
['Los guindos', '4e45aaf77d8b91a06584bd40']
['Los guindos', '4d8f35f51716a143c5c244f7']
['Los guindos', '4e40f556d164f2d277bd6ba9']
['Los guindos', '4e6dcbe7b61cf2b03c11b666']
['Los guindos', '4c42ecbcff711b8d15341405']
['Los guindos', '4d749bfc0a22236ab0cc3702']
['Los guindos', '5309f250498e04ed32f8368b']
['Los guindos', '4eac7857b6343396875a70b2']
['Los guindos', '5083fa03e4b0e99d0c467da8']
['Los guindos', '4dd903cd45ddced820ad28ae']
['Los guindos', '51201367e4b0b8daea4aab4c']
['Los guindos', '4d67b4f12433a14344364fe0']
['Los guindos', '4d920560d7b1236a4a683d38']
['R.e.n.f.e.', '50fffe6ee0e2c5d3847f28f6']
['R.e.n.f.e.', '52944e31498e7e1cc6e20543']
['R.e.n.f.e.', '4b87d100f964a5206bcd31e3']
['R.e.n.f.e.', '4d97935ae07ea35d207bdd02']
['R.e.n.f.e.', '4bb503a20fb7b713f7d2bf8a']
['R.e.n.f.e.', '4e876da85c5cfd1dcc657cea']
['R.e.n.f.e.', '4d2bedb8ebacb1f7775710

In [26]:
malaga_notas3.to_csv('mlg_notas3.csv')

In [157]:
malaga_notas

,Barrio,Latitud_barrio,Longitud_barrio,IdRest,Lugar,Latitud_lugar,Longitud_lugar,Nota,Likes,Categoría
0,Ensanche centro,36.717007,-4.420590,5690d277498e979c7ef1b953,Santa Canela,36.716856,-4.423319,8.5,38,Breakfast Spot
1,Ensanche centro,36.717007,-4.420590,58e40f20326c5a26e154a24c,Kirei,36.717125,-4.420753,8.0,8,Sushi Restaurant
2,Ensanche centro,36.717007,-4.420590,557ec82a498e1a9642eb7421,Nacalú,36.719370,-4.420385,8.3,9,Tapas Restaurant
3,Ensanche centro,36.717007,-4.420590,4c25e727136d20a13754e461,Marisquería Los Mellizos,36.718753,-4.420868,7.9,106,Seafood Restaurant
4,Ensanche centro,36.717007,-4.420590,4c68563531ba2d7f8f85f772,La Reserva 12,36.719314,-4.420329,8.2,63,Spanish Restaurant
5,Ensanche centro,36.717007,-4.420590,4b9cbd43f964a5201b7936e3,Lo Güeno de Strachan,36.719673,-4.420383,8.5,37,Tapas Restaurant
6,Ensanche centro,36.717007,-4.420590,596cf0771fa763356791342a,Brunchit España,36.718786,-4.423532,8.8,78,Restaurant
7,Ensanche centro,36.717007,-4.420590,4b767cf3f964a520994e2ee3,Casa Aranda,36.718871,-4.423368,8.7,273,Café
8,Ensanche centro,36.717007,-4.420590,51407a76e4b00ccf35393df7,El Refectorium Catedral,36.719778,-4.418763,8.7,62,Spanish Restaurant
9,Ensanche centro,36.717007,-4.420590,4e26d382d16486291ba3a75d,Al Yamal - الجمال (Al Yamal),36.715867,-4.422333,7.7,17,Moroccan Restaurant


In [171]:
malaga_notas.groupby("Categoría").mean()

,Latitud_barrio,Longitud_barrio,Latitud_lugar,Longitud_lugar,Nota,Likes
Categoría,,,,,,
Argentinian Restaurant,36.720720,-4.421059,36.722457,-4.420482,8.100000,17.000000
Bistro,36.719235,-4.420872,36.720086,-4.422666,8.060000,19.400000
Breakfast Spot,36.718863,-4.420825,36.718486,-4.421762,8.225000,48.500000
Burger Joint,36.717007,-4.420590,36.720460,-4.421305,7.200000,33.000000
Café,36.719895,-4.420955,36.720751,-4.422263,8.188889,94.000000
Comfort Food Restaurant,36.720720,-4.421059,36.721070,-4.418034,8.400000,13.000000
Deli / Bodega,36.720720,-4.421059,36.721830,-4.422827,8.350000,75.500000
Diner,36.720720,-4.421059,36.721816,-4.420671,7.600000,16.000000
Donut Shop,36.720720,-4.421059,36.722021,-4.419306,7.900000,26.000000
